In [55]:
import pandas as pd
import pyarrow as pa
import json

In [56]:
tr_targets = pd.read_csv('train_targets.csv')
all_events = pd.read_csv("all_events.csv")
video = pd.read_csv("video_info_v3.csv")
tr_targets

,viewer_uid,age,sex,age_class
0,10087154,30,male,1
1,10908708,25,female,1
2,10190464,34,male,2
3,10939673,25,male,1
4,10288257,48,male,3
...,...,...,...,...
180007,10237402,43,female,3
180008,10796936,32,male,2
180009,10684284,28,male,1
180010,10031093,50,female,3


In [57]:
train = all_events

In [58]:
train['region'] = train['region'].replace(r"[’'`]", '', regex=True)

In [59]:
train = train.merge(video, how='inner', on='rutube_video_id')

In [60]:
from datetime import timedelta

train['event_timestamp'] = pd.to_datetime(train['event_timestamp'])
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,title,category,duration,author_id,label
0,2024-06-01 13:08:30+03:00,Tatarstan Republic,smartphone,browser,Android,Chrome Mobile,60,video_395879,10813370,ТАЙНА ДВОРЦОВОГО МОСТА В ПЕТЕРБУРГЕ | А ЧТО ПО...,Лайфстайл,333313,1012572,30
1,2024-06-01 14:30:00+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,60,video_216518,10512324,"Я художник, я так вижу. Мужское / Женское. Вып...",Телепередачи,2500274,1009219,31
2,2024-06-01 18:48:12+03:00,Novosibirsk Oblast,smartphone,mobile app,Android,Rutube,121,video_41225,10951137,ДОРАМЫ ПРО ЛЮБОВЬ ♥ Романтические Фильмы ♥ Кор...,Фильмы,257997,1023998,30
3,2024-06-01 16:32:36+03:00,Moscow,desktop,browser,Windows,Yandex Browser,2324,video_215886,10912434,Менталист / The Mentalist – 4 сезон 13 серия,Сериалы,2363393,1074854,32
4,2024-06-01 00:57:04+03:00,Moscow Oblast,smartphone,mobile app,Android,Rutube,6830,video_43631,10223585,Сериал Тёмная материя – 1 сезон 4 серия / Dark...,Сериалы,3589376,1036095,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8439619,2024-06-30 18:12:06+03:00,Nizhny Novgorod Oblast,smartphone,mobile app,Android,Rutube,1735,video_163270,10687999,СЕЛЯНКА-ГОРОЖАНКА | ПРЕМЬЕРА | 1 СЕРИЯ,Телепередачи,4265440,1004920,43
8439620,2024-06-30 09:23:08+03:00,Chelyabinsk,smartphone,mobile app,Android,Rutube,4766,video_433627,10425804,"Последний герой. Русский сезон, 1 выпуск",Телепередачи,6965778,1002137,31
8439621,2024-06-30 10:57:36+03:00,Krasnodar Krai,smartphone,mobile app,Android,Rutube,890,video_11911,10565147,"Путешествие к бессмертию, 108 серия",Аниме,1111553,1028890,40
8439622,2024-06-30 18:16:55+03:00,Moscow,smartphone,browser,Android,Yandex Browser,2504,video_252573,10715847,911 Служба Спасения 4 сезон 9 серия,Сериалы,2585072,1058111,8


In [61]:
train['event_timestamp'] = train['event_timestamp'] - timedelta(hours=3)

In [62]:
time_zones = {
    'Tatarstan Republic': 3,
    'Bashkortostan Republic': 5,
    'Novosibirsk Oblast': 7,
    'Moscow': 3,
    'Moscow Oblast': 3,
    'Omsk Oblast': 7,
    'Chuvashia': 3,
    'Krasnoyarsk Krai': 7,
    'Chelyabinsk': 5,
    'Kamchatka': 11,
    'Nizhny Novgorod Oblast': 3,
    'Krasnodar Krai': 3,
    'Volgograd Oblast': 3,
    'Kaliningrad Oblast': 2,
    'Kuzbass': 7,
    'Stavropol Kray': 3,
    'Samara Oblast': 4,
    'Amur Oblast': 9,
    'Sverdlovsk Oblast': 5,
    'St.-Petersburg': 3,
    'Yamalo-Nenets': 5,
    'Orenburg Oblast': 5,
    'Khanty-Mansia': 5,
    'Kaluga Oblast': 3,
    'Tomsk Oblast': 7,
    'Novgorod Oblast': 3,
    'Arkhangelskaya': 3,
    'North Ossetia–Alania': 3,
    'Kursk Oblast': 3,
    "Leningradskaya Oblast'": 3,
    'Krasnoyarskiy': 7,
    'Ivanovo Oblast': 3,
    'Altay Kray': 7,
    'Kurgan Oblast': 5,
    'Kostroma Oblast': 3,
    'Bryansk Oblast': 3,
    'Dagestan': 3,
    'Lipetsk Oblast': 3,
    'Vladimir Oblast': 3,
    'Kirov Oblast': 3,
    'Khabarovsk': 9,
    'Tambov Oblast': 3,
    'Chukotka': 12,
    'Voronezh Oblast': 3,
    'Sverdlovsk': 5,
    'Tula Oblast': 3,
    'Krasnodarskiy': 3,
    'Irkutsk Oblast': 8,
    'Saratov Oblast': 4,
    'Khakasiya Republic': 7,
    'Penza': 3,
    'Perm Krai': 5,
    'Oryol oblast': 3,
    'Vladimir': 3,
    'Smolensk Oblast': 3,
    'Penza Oblast': 3,
    'Mordoviya Republic': 3,
    'Tyumen’ Oblast': 5,
    'Sakha': 9,
    'Primorye': 10,
    'Zabaykalskiy (Transbaikal) Kray': 9,
    'Vologda Oblast': 3,
    'Yaroslavl Oblast': 3,
    'Crimea': 3,
    'Rostov': 3,
    'Ryazan Oblast': 3,
    'Perm': 5,
    'Chechnya': 3,
    'Udmurtiya Republic': 3,
    'Tver Oblast': 3,
    'Buryatiya Republic': 8,
    'Belgorod Oblast': 3,
    'Kaluga': 3,
    'Astrakhan Oblast': 3,
    'Karelia': 3,
    'Murmansk': 3,
    'Adygeya Republic': 3,
    'Kemerovo Oblast': 7,
    'Mariy-El Republic': 3,
    'Kursk': 3,
    'Saratovskaya Oblast': 4,
    'Sakhalin Oblast': 10,
    'Ivanovo': 3,
    'Tyumen Oblast': 5,
    'Stavropol’ Kray': 3,
    'Voronezj': 3,
    'Karachayevo-Cherkesiya Republic': 3,
    'Kabardino-Balkariya Republic': 3,
    'Ulyanovsk': 4,
    'North Ossetia': 3,
    'Komi': 3,
    'Smolensk': 3,
    'Tver’ Oblast': 3,
    'Sebastopol City': 3,
    'Pskov Oblast': 3,
    'Tula': 3,
    'Orel Oblast': 3,
    'Jaroslavl': 3,
    'Tambov': 3,
    'Kalmykiya Republic': 3,
    'Primorskiy (Maritime) Kray': 10,
    'Altai': 7,
    'Magadan Oblast': 10,
    'Vologda': 3,
    'Tyva Republic': 7,
    'Nenets': 3,
    'Smolenskaya Oblast': 3,
    'Jewish Autonomous Oblast': 10,
    'Astrakhan': 3,
    'Ingushetiya Republic': 3,
    'Kirov': 3,
    'Transbaikal Territory': 9,
    'Omsk': 7,
    'Kaliningrad': 2,
    'Stavropol Krai': 3,
    'Arkhangelsk Oblast': 3,
    'Leningradskaya Oblast': 3
}

In [63]:
train['timezone_offset'] = train['region'].map(time_zones)

In [64]:
train['event_timestamp_timezone'] = train['event_timestamp'] + pd.to_timedelta(train['timezone_offset'], unit='h')

In [65]:
train['morning'] = 0
train['day'] = 0
train['evening'] = 0
train['night'] = 0
train.loc[train['event_timestamp_timezone'].dt.hour.between(6, 12, inclusive='left'), 'morning'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(13, 18, inclusive='left'), 'day'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(19, 23, inclusive='left'), 'evening'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(0, 5, inclusive='both'), 'night'] = 1

In [66]:
with open("category.json", 'r') as f:
    cat_dict = json.loads(f.read())
train['category'] = train['category'].map(cat_dict)

In [67]:
with open("regions.json", 'r') as file:
    region_dict = json.loads(file.read())
train['region'] = train['region'].map(region_dict)
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,title,category,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night
0,2024-06-01 10:08:30+03:00,0,smartphone,browser,Android,Chrome Mobile,60,video_395879,10813370,ТАЙНА ДВОРЦОВОГО МОСТА В ПЕТЕРБУРГЕ | А ЧТО ПО...,13,333313,1012572,30,3,2024-06-01 13:08:30+03:00,0,1,0,0
1,2024-06-01 11:30:00+03:00,1,smartphone,mobile app,Android,Rutube,60,video_216518,10512324,"Я художник, я так вижу. Мужское / Женское. Вып...",16,2500274,1009219,31,5,2024-06-01 16:30:00+03:00,0,1,0,0
2,2024-06-01 15:48:12+03:00,2,smartphone,mobile app,Android,Rutube,121,video_41225,10951137,ДОРАМЫ ПРО ЛЮБОВЬ ♥ Романтические Фильмы ♥ Кор...,2,257997,1023998,30,7,2024-06-01 22:48:12+03:00,0,0,1,0
3,2024-06-01 13:32:36+03:00,3,desktop,browser,Windows,Yandex Browser,2324,video_215886,10912434,Менталист / The Mentalist – 4 сезон 13 серия,1,2363393,1074854,32,3,2024-06-01 16:32:36+03:00,0,1,0,0
4,2024-05-31 21:57:04+03:00,4,smartphone,mobile app,Android,Rutube,6830,video_43631,10223585,Сериал Тёмная материя – 1 сезон 4 серия / Dark...,1,3589376,1036095,8,3,2024-06-01 00:57:04+03:00,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8439619,2024-06-30 15:12:06+03:00,10,smartphone,mobile app,Android,Rutube,1735,video_163270,10687999,СЕЛЯНКА-ГОРОЖАНКА | ПРЕМЬЕРА | 1 СЕРИЯ,16,4265440,1004920,43,3,2024-06-30 18:12:06+03:00,0,0,0,0
8439620,2024-06-30 06:23:08+03:00,8,smartphone,mobile app,Android,Rutube,4766,video_433627,10425804,"Последний герой. Русский сезон, 1 выпуск",16,6965778,1002137,31,5,2024-06-30 11:23:08+03:00,1,0,0,0
8439621,2024-06-30 07:57:36+03:00,11,smartphone,mobile app,Android,Rutube,890,video_11911,10565147,"Путешествие к бессмертию, 108 серия",5,1111553,1028890,40,3,2024-06-30 10:57:36+03:00,1,0,0,0
8439622,2024-06-30 15:16:55+03:00,3,smartphone,browser,Android,Yandex Browser,2504,video_252573,10715847,911 Служба Спасения 4 сезон 9 серия,1,2585072,1058111,8,3,2024-06-30 18:16:55+03:00,0,0,0,0


In [68]:
train['duration'] = train['duration'] / 1000
train['percent'] = train['total_watchtime'] / train['duration']

In [69]:
train = train.drop(['title'], axis=1)

In [70]:
mn = train['ua_client_type'].value_counts().idxmax()
train['ua_client_type'] = train['ua_client_type'].fillna(mn)

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,category,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 10:08:30+03:00,0,smartphone,browser,Android,Chrome Mobile,60,video_395879,10813370,13,333.313,1012572,30,3,2024-06-01 13:08:30+03:00,0,1,0,0,0.180011
1,2024-06-01 11:30:00+03:00,1,smartphone,mobile app,Android,Rutube,60,video_216518,10512324,16,2500.274,1009219,31,5,2024-06-01 16:30:00+03:00,0,1,0,0,0.023997
2,2024-06-01 15:48:12+03:00,2,smartphone,mobile app,Android,Rutube,121,video_41225,10951137,2,257.997,1023998,30,7,2024-06-01 22:48:12+03:00,0,0,1,0,0.468998
3,2024-06-01 13:32:36+03:00,3,desktop,browser,Windows,Yandex Browser,2324,video_215886,10912434,1,2363.393,1074854,32,3,2024-06-01 16:32:36+03:00,0,1,0,0,0.983332
4,2024-05-31 21:57:04+03:00,4,smartphone,mobile app,Android,Rutube,6830,video_43631,10223585,1,3589.376,1036095,8,3,2024-06-01 00:57:04+03:00,0,0,0,1,1.902838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8439619,2024-06-30 15:12:06+03:00,10,smartphone,mobile app,Android,Rutube,1735,video_163270,10687999,16,4265.440,1004920,43,3,2024-06-30 18:12:06+03:00,0,0,0,0,0.406758
8439620,2024-06-30 06:23:08+03:00,8,smartphone,mobile app,Android,Rutube,4766,video_433627,10425804,16,6965.778,1002137,31,5,2024-06-30 11:23:08+03:00,1,0,0,0,0.684202
8439621,2024-06-30 07:57:36+03:00,11,smartphone,mobile app,Android,Rutube,890,video_11911,10565147,5,1111.553,1028890,40,3,2024-06-30 10:57:36+03:00,1,0,0,0,0.800682
8439622,2024-06-30 15:16:55+03:00,3,smartphone,browser,Android,Yandex Browser,2504,video_252573,10715847,1,2585.072,1058111,8,3,2024-06-30 18:16:55+03:00,0,0,0,0,0.968638


In [71]:
with open("client_types.json", 'r') as file:
    client_type_dict = json.loads(file.read())
train['ua_client_type'] = train['ua_client_type'].map(client_type_dict)

In [72]:
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,category,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 10:08:30+03:00,0,smartphone,0,Android,Chrome Mobile,60,video_395879,10813370,13,333.313,1012572,30,3,2024-06-01 13:08:30+03:00,0,1,0,0,0.180011
1,2024-06-01 11:30:00+03:00,1,smartphone,1,Android,Rutube,60,video_216518,10512324,16,2500.274,1009219,31,5,2024-06-01 16:30:00+03:00,0,1,0,0,0.023997
2,2024-06-01 15:48:12+03:00,2,smartphone,1,Android,Rutube,121,video_41225,10951137,2,257.997,1023998,30,7,2024-06-01 22:48:12+03:00,0,0,1,0,0.468998
3,2024-06-01 13:32:36+03:00,3,desktop,0,Windows,Yandex Browser,2324,video_215886,10912434,1,2363.393,1074854,32,3,2024-06-01 16:32:36+03:00,0,1,0,0,0.983332
4,2024-05-31 21:57:04+03:00,4,smartphone,1,Android,Rutube,6830,video_43631,10223585,1,3589.376,1036095,8,3,2024-06-01 00:57:04+03:00,0,0,0,1,1.902838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8439619,2024-06-30 15:12:06+03:00,10,smartphone,1,Android,Rutube,1735,video_163270,10687999,16,4265.440,1004920,43,3,2024-06-30 18:12:06+03:00,0,0,0,0,0.406758
8439620,2024-06-30 06:23:08+03:00,8,smartphone,1,Android,Rutube,4766,video_433627,10425804,16,6965.778,1002137,31,5,2024-06-30 11:23:08+03:00,1,0,0,0,0.684202
8439621,2024-06-30 07:57:36+03:00,11,smartphone,1,Android,Rutube,890,video_11911,10565147,5,1111.553,1028890,40,3,2024-06-30 10:57:36+03:00,1,0,0,0,0.800682
8439622,2024-06-30 15:16:55+03:00,3,smartphone,0,Android,Yandex Browser,2504,video_252573,10715847,1,2585.072,1058111,8,3,2024-06-30 18:16:55+03:00,0,0,0,0,0.968638


In [74]:
mn = train['ua_os'].value_counts().idxmax()
train['ua_os'] = train['ua_os'].fillna(mn)

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,category,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 10:08:30+03:00,0,smartphone,0,Android,Chrome Mobile,60,video_395879,10813370,13,333.313,1012572,30,3,2024-06-01 13:08:30+03:00,0,1,0,0,0.180011
1,2024-06-01 11:30:00+03:00,1,smartphone,1,Android,Rutube,60,video_216518,10512324,16,2500.274,1009219,31,5,2024-06-01 16:30:00+03:00,0,1,0,0,0.023997
2,2024-06-01 15:48:12+03:00,2,smartphone,1,Android,Rutube,121,video_41225,10951137,2,257.997,1023998,30,7,2024-06-01 22:48:12+03:00,0,0,1,0,0.468998
3,2024-06-01 13:32:36+03:00,3,desktop,0,Windows,Yandex Browser,2324,video_215886,10912434,1,2363.393,1074854,32,3,2024-06-01 16:32:36+03:00,0,1,0,0,0.983332
4,2024-05-31 21:57:04+03:00,4,smartphone,1,Android,Rutube,6830,video_43631,10223585,1,3589.376,1036095,8,3,2024-06-01 00:57:04+03:00,0,0,0,1,1.902838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8439619,2024-06-30 15:12:06+03:00,10,smartphone,1,Android,Rutube,1735,video_163270,10687999,16,4265.440,1004920,43,3,2024-06-30 18:12:06+03:00,0,0,0,0,0.406758
8439620,2024-06-30 06:23:08+03:00,8,smartphone,1,Android,Rutube,4766,video_433627,10425804,16,6965.778,1002137,31,5,2024-06-30 11:23:08+03:00,1,0,0,0,0.684202
8439621,2024-06-30 07:57:36+03:00,11,smartphone,1,Android,Rutube,890,video_11911,10565147,5,1111.553,1028890,40,3,2024-06-30 10:57:36+03:00,1,0,0,0,0.800682
8439622,2024-06-30 15:16:55+03:00,3,smartphone,0,Android,Yandex Browser,2504,video_252573,10715847,1,2585.072,1058111,8,3,2024-06-30 18:16:55+03:00,0,0,0,0,0.968638


In [75]:
with open('os.json', 'r') as file:
    os_dict = json.loads(file.read())
train['ua_os'] = train['ua_os'].map(os_dict)
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,category,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 10:08:30+03:00,0,smartphone,0,0,Chrome Mobile,60,video_395879,10813370,13,333.313,1012572,30,3,2024-06-01 13:08:30+03:00,0,1,0,0,0.180011
1,2024-06-01 11:30:00+03:00,1,smartphone,1,0,Rutube,60,video_216518,10512324,16,2500.274,1009219,31,5,2024-06-01 16:30:00+03:00,0,1,0,0,0.023997
2,2024-06-01 15:48:12+03:00,2,smartphone,1,0,Rutube,121,video_41225,10951137,2,257.997,1023998,30,7,2024-06-01 22:48:12+03:00,0,0,1,0,0.468998
3,2024-06-01 13:32:36+03:00,3,desktop,0,1,Yandex Browser,2324,video_215886,10912434,1,2363.393,1074854,32,3,2024-06-01 16:32:36+03:00,0,1,0,0,0.983332
4,2024-05-31 21:57:04+03:00,4,smartphone,1,0,Rutube,6830,video_43631,10223585,1,3589.376,1036095,8,3,2024-06-01 00:57:04+03:00,0,0,0,1,1.902838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8439619,2024-06-30 15:12:06+03:00,10,smartphone,1,0,Rutube,1735,video_163270,10687999,16,4265.440,1004920,43,3,2024-06-30 18:12:06+03:00,0,0,0,0,0.406758
8439620,2024-06-30 06:23:08+03:00,8,smartphone,1,0,Rutube,4766,video_433627,10425804,16,6965.778,1002137,31,5,2024-06-30 11:23:08+03:00,1,0,0,0,0.684202
8439621,2024-06-30 07:57:36+03:00,11,smartphone,1,0,Rutube,890,video_11911,10565147,5,1111.553,1028890,40,3,2024-06-30 10:57:36+03:00,1,0,0,0,0.800682
8439622,2024-06-30 15:16:55+03:00,3,smartphone,0,0,Yandex Browser,2504,video_252573,10715847,1,2585.072,1058111,8,3,2024-06-30 18:16:55+03:00,0,0,0,0,0.968638


In [76]:
mn = train['ua_client_name'].value_counts().idxmax()
train['ua_client_name'] = train['ua_client_name'].fillna(mn)

In [78]:
with open("client_names.json", 'r') as file:
    client_name_dict = json.loads(file.read())
train['ua_client_name'] = train['ua_client_name'].map(client_name_dict)


In [80]:
with open("devices.json", 'r') as file:
    device_dict = json.loads(file.read())
train['ua_device_type'] = train['ua_device_type'].map(device_dict)

In [81]:
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,category,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 10:08:30+03:00,0,1,0,0,0,60,video_395879,10813370,13,333.313,1012572,30,3,2024-06-01 13:08:30+03:00,0,1,0,0,0.180011
1,2024-06-01 11:30:00+03:00,1,1,1,0,1,60,video_216518,10512324,16,2500.274,1009219,31,5,2024-06-01 16:30:00+03:00,0,1,0,0,0.023997
2,2024-06-01 15:48:12+03:00,2,1,1,0,1,121,video_41225,10951137,2,257.997,1023998,30,7,2024-06-01 22:48:12+03:00,0,0,1,0,0.468998
3,2024-06-01 13:32:36+03:00,3,0,0,1,2,2324,video_215886,10912434,1,2363.393,1074854,32,3,2024-06-01 16:32:36+03:00,0,1,0,0,0.983332
4,2024-05-31 21:57:04+03:00,4,1,1,0,1,6830,video_43631,10223585,1,3589.376,1036095,8,3,2024-06-01 00:57:04+03:00,0,0,0,1,1.902838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8439619,2024-06-30 15:12:06+03:00,10,1,1,0,1,1735,video_163270,10687999,16,4265.440,1004920,43,3,2024-06-30 18:12:06+03:00,0,0,0,0,0.406758
8439620,2024-06-30 06:23:08+03:00,8,1,1,0,1,4766,video_433627,10425804,16,6965.778,1002137,31,5,2024-06-30 11:23:08+03:00,1,0,0,0,0.684202
8439621,2024-06-30 07:57:36+03:00,11,1,1,0,1,890,video_11911,10565147,5,1111.553,1028890,40,3,2024-06-30 10:57:36+03:00,1,0,0,0,0.800682
8439622,2024-06-30 15:16:55+03:00,3,1,0,0,2,2504,video_252573,10715847,1,2585.072,1058111,8,3,2024-06-30 18:16:55+03:00,0,0,0,0,0.968638


In [82]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, max_iter=1000).fit(train[['ua_device_type', 'ua_client_type', 'ua_os', 'ua_client_name']])

In [83]:
train['device'] = kmeans.labels_

In [85]:
%%time
def get_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None

aggregated = train.groupby('viewer_uid').agg({
    'event_timestamp_timezone': ['count', 'min', 'max', 'mean'],
    'region': get_mode,
    'label': get_mode,
    'category': get_mode,
    'total_watchtime': ['mean', 'max', 'min', 'sum'],
    'device': get_mode,
    'author_id': get_mode,
    'rutube_video_id': get_mode,
    'percent': ['mean', 'min', 'max', 'sum'],
    'duration': ['min', 'max', 'sum', 'mean']
})
aggregated

CPU times: total: 12min 54s
Wall time: 13min


event_timestamp                            \
                     count                       min   
viewer_uid                                             
10000000                 1 2024-06-10 03:39:18+03:00   
10000003                27 2024-06-01 17:18:49+03:00   
10000013                 1 2024-06-15 07:38:17+03:00   
10000015                 1 2024-06-19 16:37:07+03:00   
10000022                 5 2024-06-15 07:34:58+03:00   
...                    ...                       ...   
11140892                 1 2024-06-30 20:53:58+03:00   
11140893                 1 2024-06-30 20:57:29+03:00   
11140894                 1 2024-06-30 20:55:09+03:00   
11140895                 1 2024-06-30 20:55:16+03:00   
11140896                 1 2024-06-30 20:56:00+03:00   

                                                                          \
                                 max                                mean   
viewer_uid                                                                 
10000000   2024-06-10 03:39:18+03:00           2024-06-10 03:39:18+03:00   
10000003   2024-06-30 20:24:38+03:00 2024-06-17 11:16:39.370370304+03:00   
10000013   2024-06-15 07:38:17+03:00           2024-06-15 07:38:17+03:00   
10000015   2024-06-19 16:37:07+03:00           2024-06-19 16:37:07+03:00   
10000022   2024-06-27 06:05:34+03:00    2024-06-18 09:30:08.400000+03:00   
...                              ...                                 ...   
11140892   2024-06-30 20:53:58+03:00           2024-06-30 20:53:58+03:00   
11140893   2024-06-30 20:57:29+03:00           2024-06-30 20:57:29+03:00   
11140894   2024-06-30 20:55:09+03:00           2024-06-30 20:55:09+03:00   
11140895   2024-06-30 20:55:16+03:00           2024-06-30 20:55:16+03:00   
11140896   2024-06-30 20:56:00+03:00           2024-06-30 20:56:00+03:00   

             region    label category total_watchtime              ...  \
           get_mode get_mode get_mode            mean    max  min  ...   
viewer_uid                                                         ...   
10000000          3       30        4      101.000000    101  101  ...   
10000003          4        8        1     3033.296296  10432  981  ...   
10000013          0       30        4      722.000000    722  722  ...   
10000015         10       47       29       82.000000     82   82  ...   
10000022          4       30        4      651.200000   2261   40  ...   
...             ...      ...      ...             ...    ...  ...  ...   
11140892         11       44        1      302.000000    302  302  ...   
11140893         51       22       11      121.000000    121  121  ...   
11140894         48       40        2      283.000000    283  283  ...   
11140895         15        8        1      269.000000    269  269  ...   
11140896          3       30       27      223.000000    223  223  ...   

           author_id rutube_video_id   percent                                 \
            get_mode        get_mode      mean       min       max        sum   
viewer_uid                                                                      
10000000     1059748    video_100053  0.173571  0.173571  0.173571   0.173571   
10000003     1036141    video_399250  0.836207  0.290536  0.963947  22.577582   
10000013     1009406     video_44766  0.983490  0.983490  0.983490   0.983490   
10000015     1006760    video_311154  0.194807  0.194807  0.194807   0.194807   
10000022     1000004     video_13816  0.576738  0.124781  0.999909   2.883689   
...              ...             ...       ...       ...       ...        ...   
11140892     1037886    video_450725  0.311088  0.311088  0.311088   0.311088   
11140893     1028705    video_327678  0.124037  0.124037  0.124037   0.124037   
11140894     1013331    video_451186  0.591555  0.591555  0.591555   0.591555   
11140895     1102087    video_112377  0.108949  0.108949  0.108949   0.108949   
11140896     1012907    video_407154  0.164430  0.164430  0.164430   0.

In [86]:
aggregated = aggregated.reset_index(drop=False)

In [87]:
aggregated.columns = ['_'.join(filter(None, col)).strip() for col in aggregated.columns]
aggregated.columns

Index(['viewer_uid', 'event_timestamp_count', 'event_timestamp_min',
       'event_timestamp_max', 'event_timestamp_mean', 'region_get_mode',
       'label_get_mode', 'category_get_mode', 'total_watchtime_mean',
       'total_watchtime_max', 'total_watchtime_min', 'total_watchtime_sum',
       'device_get_mode', 'author_id_get_mode', 'rutube_video_id_get_mode',
       'percent_mean', 'percent_min', 'percent_max', 'percent_sum',
       'duration_min', 'duration_max', 'duration_sum', 'duration_mean'],
      dtype='object')

In [88]:
aggregated['event_timestamp_timezone_min'] = aggregated['event_timestamp_min'].apply(lambda x: x.timestamp())
aggregated['event_timestamp_timezone_max'] = aggregated['event_timestamp_max'].apply(lambda x: x.timestamp())
aggregated['event_timestamp_timezone_mean'] = aggregated['event_timestamp_mean'].apply(lambda x: x.timestamp())
aggregated

,viewer_uid,event_timestamp_count,event_timestamp_min,event_timestamp_max,event_timestamp_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,author_id_get_mode,rutube_video_id_get_mode,percent_mean,percent_min,percent_max,percent_sum,duration_min,duration_max,duration_sum,duration_mean
0,10000000,1,1.717980e+09,1.717980e+09,1.717980e+09,3,30,4,101.000000,101,...,1059748,video_100053,0.173571,0.173571,0.173571,0.173571,581.893,581.893,581.893,581.893000
1,10000003,27,1.717252e+09,1.719768e+09,1.718612e+09,4,8,1,3033.296296,10432,...,1036141,video_399250,0.836207,0.290536,0.963947,22.577582,2684.292,10822.176,97396.926,3607.293556
2,10000013,1,1.718426e+09,1.718426e+09,1.718426e+09,0,30,4,722.000000,722,...,1009406,video_44766,0.983490,0.983490,0.983490,0.983490,734.120,734.120,734.120,734.120000
3,10000015,1,1.718804e+09,1.718804e+09,1.718804e+09,10,47,29,82.000000,82,...,1006760,video_311154,0.194807,0.194807,0.194807,0.194807,420.929,420.929,420.929,420.929000
4,10000022,5,1.718426e+09,1.719458e+09,1.718692e+09,4,30,4,651.200000,2261,...,1000004,video_13816,0.576738,0.124781,0.999909,2.883689,195.371,2261.206,4044.406,808.881200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960880,11140892,1,1.719770e+09,1.719770e+09,1.719770e+09,11,44,1,302.000000,302,...,1037886,video_450725,0.311088,0.311088,0.311088,0.311088,970.785,970.785,970.785,970.785000
960881,11140893,1,1.719770e+09,1.719770e+09,1.719770e+09,51,22,11,121.000000,121,...,1028705,video_327678,0.124037,0.124037,0.124037,0.124037,975.517,975.517,975.517,975.517000
960882,11140894,1,1.719770e+09,1.719770e+09,1.719770e+09,48,40,2,283.000000,283,...,1013331,video_451186,0.591555,0.591555,0.591555,0.591555,478.400,478.400,478.400,478.400000
960883,11140895,1,1.719770e+09,1.719770e+09,1.719770e+09,15,8,1,269.000000,269,...,1102087,video_112377,0.108949,0.108949,0.108949,0.108949,2469.056,2469.056,2469.056,2469.056000


In [89]:
t = pa.Table.from_pandas(train)

In [90]:
time_sum = t.select(['viewer_uid', 'morning', 'day','evening', 'night']).group_by(['viewer_uid']).aggregate([('morning', "sum"), ('day', "sum"), ('evening', "sum"), ('night', "sum")])
time_sum = time_sum.to_pandas()

In [91]:
aggregated = aggregated.merge(time_sum, how='inner', on='viewer_uid')
aggregated

,viewer_uid,event_timestamp_count,event_timestamp_min,event_timestamp_max,event_timestamp_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,percent_max,percent_sum,duration_min,duration_max,duration_sum,duration_mean,morning_sum,day_sum,evening_sum,night_sum
0,10000000,1,1.717980e+09,1.717980e+09,1.717980e+09,3,30,4,101.000000,101,...,0.173571,0.173571,581.893,581.893,581.893,581.893000,1,0,0,0
1,10000003,27,1.717252e+09,1.719768e+09,1.718612e+09,4,8,1,3033.296296,10432,...,0.963947,22.577582,2684.292,10822.176,97396.926,3607.293556,0,0,19,0
2,10000013,1,1.718426e+09,1.718426e+09,1.718426e+09,0,30,4,722.000000,722,...,0.983490,0.983490,734.120,734.120,734.120,734.120000,1,0,0,0
3,10000015,1,1.718804e+09,1.718804e+09,1.718804e+09,10,47,29,82.000000,82,...,0.194807,0.194807,420.929,420.929,420.929,420.929000,0,0,1,0
4,10000022,5,1.718426e+09,1.719458e+09,1.718692e+09,4,30,4,651.200000,2261,...,0.999909,2.883689,195.371,2261.206,4044.406,808.881200,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960880,11140892,1,1.719770e+09,1.719770e+09,1.719770e+09,11,44,1,302.000000,302,...,0.311088,0.311088,970.785,970.785,970.785,970.785000,0,0,0,0
960881,11140893,1,1.719770e+09,1.719770e+09,1.719770e+09,51,22,11,121.000000,121,...,0.124037,0.124037,975.517,975.517,975.517,975.517000,0,0,0,1
960882,11140894,1,1.719770e+09,1.719770e+09,1.719770e+09,48,40,2,283.000000,283,...,0.591555,0.591555,478.400,478.400,478.400,478.400000,0,0,0,1
960883,11140895,1,1.719770e+09,1.719770e+09,1.719770e+09,15,8,1,269.000000,269,...,0.108949,0.108949,2469.056,2469.056,2469.056,2469.056000,0,0,0,0


In [92]:
videos = video['rutube_video_id'].unique()
video_dict = {v: v_id for v_id, v in enumerate(videos)}

In [93]:
aggregated['rutube_video_id_get_mode'] =  aggregated['rutube_video_id_get_mode'].map(video_dict)

In [94]:
aggregated

,viewer_uid,event_timestamp_count,event_timestamp_min,event_timestamp_max,event_timestamp_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,percent_max,percent_sum,duration_min,duration_max,duration_sum,duration_mean,morning_sum,day_sum,evening_sum,night_sum
0,10000000,1,1.717980e+09,1.717980e+09,1.717980e+09,3,30,4,101.000000,101,...,0.173571,0.173571,581.893,581.893,581.893,581.893000,1,0,0,0
1,10000003,27,1.717252e+09,1.719768e+09,1.718612e+09,4,8,1,3033.296296,10432,...,0.963947,22.577582,2684.292,10822.176,97396.926,3607.293556,0,0,19,0
2,10000013,1,1.718426e+09,1.718426e+09,1.718426e+09,0,30,4,722.000000,722,...,0.983490,0.983490,734.120,734.120,734.120,734.120000,1,0,0,0
3,10000015,1,1.718804e+09,1.718804e+09,1.718804e+09,10,47,29,82.000000,82,...,0.194807,0.194807,420.929,420.929,420.929,420.929000,0,0,1,0
4,10000022,5,1.718426e+09,1.719458e+09,1.718692e+09,4,30,4,651.200000,2261,...,0.999909,2.883689,195.371,2261.206,4044.406,808.881200,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960880,11140892,1,1.719770e+09,1.719770e+09,1.719770e+09,11,44,1,302.000000,302,...,0.311088,0.311088,970.785,970.785,970.785,970.785000,0,0,0,0
960881,11140893,1,1.719770e+09,1.719770e+09,1.719770e+09,51,22,11,121.000000,121,...,0.124037,0.124037,975.517,975.517,975.517,975.517000,0,0,0,1
960882,11140894,1,1.719770e+09,1.719770e+09,1.719770e+09,48,40,2,283.000000,283,...,0.591555,0.591555,478.400,478.400,478.400,478.400000,0,0,0,1
960883,11140895,1,1.719770e+09,1.719770e+09,1.719770e+09,15,8,1,269.000000,269,...,0.108949,0.108949,2469.056,2469.056,2469.056,2469.056000,0,0,0,0


In [95]:
from catboost import CatBoostClassifier

sex_classifier = CatBoostClassifier()
sex_classifier.load_model('sex_model')

In [98]:
age_classifier = CatBoostClassifier()
age_classifier.load_model("age_model")

In [97]:
sex_classes = sex_classifier.predict(aggregated.drop(['viewer_uid'], axis=1))

In [99]:
age_classes = age_classifier.predict(aggregated.drop(['viewer_uid'], axis=1))
age_classes

array([[2],
       [1],
       [2],
       ...,
       [2],
       [2],
       [2]], dtype=int64)